In [1]:
import os
import os.path
import glob
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold                                                                                                                       
from sklearn.metrics import confusion_matrix,accuracy_score
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.model_selection import GridSearchCV
import sklearn_evaluation.plot as skplot
from sklearn.model_selection import learning_curve
from sklearn import svm

import matplotlib.pyplot as plt
import matplotlib.cm as colormap
plt.rcParams['image.cmap'] = 'Paired'

import numpy as np
np.random.seed(1)

from keras.preprocessing.image import img_to_array
from keras.utils import np_utils
from keras.preprocessing import image
from keras.applications.vgg19 import VGG19
from keras.applications.imagenet_utils import preprocess_input

os.environ['CUDA_VISIBLE_DEVICES'] = '-1'

Using TensorFlow backend.


In [2]:
imagedir = "Datasets/malimg_paper_dataset_imgs"

In [3]:
cur_dir = os.getcwd()
os.chdir(imagedir)  # the parent folder with sub-folders

# Get number of samples per family
list_fams = sorted(os.listdir(os.getcwd()), key=str.lower)  # vector of strings with family names
no_imgs = []  # No. of samples per family
for i in range(len(list_fams)):
    os.chdir(list_fams[i])
    len1 = len(glob.glob('*.png'))  # assuming the images are stored as 'png'
    no_imgs.append(len1)
    os.chdir('..')
num_samples = np.sum(no_imgs)  # total number of all samples

# Compute the labels
y = np.zeros(num_samples)
pos = 0
label = 0
for i in no_imgs:
    print ("Label:%2d\tFamily: %15s\tNumber of images: %d" % (label, list_fams[label], i))
    for j in range(i):
        y[pos] = label
        pos += 1
    label += 1
num_classes = label

# Compute the features
width, height,channels = (224,224,3)
X = np.zeros((num_samples, width, height, channels))
cnt = 0
list_paths = [] # List of image paths
print("Processing images ...")
for i in range(len(list_fams)):
    for img_file in glob.glob(list_fams[i]+'/*.png'):
        #print("[%d] Processing image: %s" % (cnt, img_file))
        list_paths.append(os.path.join(os.getcwd(),img_file))
        img = image.load_img(img_file, target_size=(224, 224))
        x = image.img_to_array(img)
        x = np.expand_dims(x, axis=0)
        x = preprocess_input(x)
        X[cnt] = x
        cnt += 1
print("Images processed: %d" %(cnt))

os.chdir(cur_dir)

Label: 0	Family:       Adialer.C	Number of images: 122
Label: 1	Family:       Agent.FYI	Number of images: 116
Label: 2	Family:       Allaple.A	Number of images: 2949
Label: 3	Family:       Allaple.L	Number of images: 1591
Label: 4	Family:   Alueron.gen!J	Number of images: 198
Label: 5	Family:       Autorun.K	Number of images: 106
Label: 6	Family:     C2LOP.gen!g	Number of images: 200
Label: 7	Family:         C2LOP.P	Number of images: 146
Label: 8	Family:  Dialplatform.B	Number of images: 177
Label: 9	Family:       Dontovo.A	Number of images: 162
Label:10	Family:        Fakerean	Number of images: 381
Label:11	Family:   Instantaccess	Number of images: 431
Label:12	Family:      Lolyda.AA1	Number of images: 213
Label:13	Family:      Lolyda.AA2	Number of images: 184
Label:14	Family:      Lolyda.AA3	Number of images: 123
Label:15	Family:       Lolyda.AT	Number of images: 159
Label:16	Family:     Malex.gen!J	Number of images: 136
Label:17	Family:   Obfuscator.AD	Number of images: 142
Label:18

In [4]:
X.shape

(9339, 224, 224, 3)

In [5]:
# Encoding classes (y) into integers (y_encoded) and then generating one-hot-encoding (Y)
encoder = LabelEncoder()
encoder.fit(y)
y_encoded = encoder.transform(y)
Y = np_utils.to_categorical(y_encoded)

In [6]:
# Creating base_model (VGG19 notop)
image_shape = (224, 224, 3)                                                                                                                                                                                                                                                                                            
base_model = VGG19(weights='imagenet', input_shape=image_shape, include_top=False)

In [7]:
filename = 'malimg-vgg19features.npy'
if os.path.exists(filename):
    print("Loading VGG19 extracted features from %s ..." %(filename))
    vgg19features = np.load(filename)
else:
    print("Extracting features from VGG19 layers ...")
    vgg19features = base_model.predict(X)
    print("Saving VGG19 extracted features into %s ..." %(filename))
    np.save(filename, vgg19features)

Loading VGG19 extracted features from malimg-vgg19features.npy ...


In [8]:
vgg19features.shape

(9339, 7, 7, 512)

In [9]:
vgg19features = np.reshape(vgg19features,(vgg19features.shape[0],-1))

In [ ]:
vgg19features.shape

(9339, 25088)

In [ ]:
# Training top_model
C_range = np.logspace(-2, 10, 13)
gamma_range = np.logspace(-9, 3, 13)
param_grid = dict(gamma=gamma_range, C=C_range)
cv = StratifiedShuffleSplit(n_splits=5, test_size=0.2, random_state=1)

top_model = GridSearchCV(svm.SVC(), param_grid=param_grid, cv=cv, verbose=10, n_jobs=15)
top_model.fit(vgg19features,y)  # Training 

Fitting 5 folds for each of 169 candidates, totalling 845 fits
[CV] C=0.01, gamma=1e-09 .............................................
[CV] C=0.01, gamma=1e-09 .............................................
[CV] C=0.01, gamma=1e-09 .............................................
[CV] C=0.01, gamma=1e-09 .............................................
[CV] C=0.01, gamma=1e-09 .............................................
[CV] C=0.01, gamma=1e-08 .............................................
[CV] C=0.01, gamma=1e-08 .............................................
[CV] C=0.01, gamma=1e-08 .............................................
[CV] C=0.01, gamma=1e-08 .............................................
[CV] C=0.01, gamma=1e-08 .............................................
[CV] C=0.01, gamma=1e-07 .............................................
[CV] C=0.01, gamma=1e-07 .............................................
[CV] C=0.01, gamma=1e-07 .............................................
[CV] C=0.01, g

[Parallel(n_jobs=15)]: Done   2 tasks      | elapsed: 116.3min


[CV] .............. C=0.01, gamma=1e-08, score=0.315846, total=74.1min
[CV] C=0.01, gamma=1e-06 .............................................
[CV] .............. C=0.01, gamma=1e-07, score=0.401499, total=74.8min
[CV] C=0.01, gamma=1e-06 .............................................
[CV] .............. C=0.01, gamma=1e-07, score=0.401499, total=75.3min
[CV] C=0.01, gamma=1e-06 .............................................
[CV] .............. C=0.01, gamma=1e-07, score=0.401499, total=75.7min
[CV] C=0.01, gamma=1e-05 .............................................
[CV] .............. C=0.01, gamma=1e-08, score=0.315846, total=79.3min
[CV] C=0.01, gamma=1e-05 .............................................
[CV] .............. C=0.01, gamma=1e-09, score=0.315846, total=79.2min
[CV] C=0.01, gamma=1e-05 .............................................
[CV] .............. C=0.01, gamma=1e-07, score=0.401499, total=78.9min
[CV] C=0.01, gamma=1e-05 .............................................
[CV] .

[Parallel(n_jobs=15)]: Done  11 tasks      | elapsed: 129.8min


[CV] .............. C=0.01, gamma=1e-08, score=0.315846, total=83.6min
[CV] C=0.01, gamma=0.0001 ............................................
[CV] .............. C=0.01, gamma=1e-07, score=0.401499, total=81.0min
[CV] C=0.01, gamma=0.0001 ............................................
[CV] .............. C=0.01, gamma=1e-09, score=0.315846, total=81.9min
[CV] C=0.01, gamma=0.0001 ............................................
[CV] .............. C=0.01, gamma=1e-08, score=0.315846, total=81.2min
[CV] C=0.01, gamma=0.0001 ............................................
[CV] .............. C=0.01, gamma=1e-06, score=0.717880, total=57.2min
[CV] C=0.01, gamma=0.001 .............................................
[CV] .............. C=0.01, gamma=1e-06, score=0.716809, total=57.0min
[CV] C=0.01, gamma=0.001 .............................................
[CV] .............. C=0.01, gamma=1e-06, score=0.718415, total=59.8min
[CV] C=0.01, gamma=0.001 .............................................
[CV] .

...

[CV] ........... C=10000.0, gamma=1e-07, score=0.988758, total= 9.8min
[CV] C=10000.0, gamma=1e-06 ..........................................
[CV] ........... C=10000.0, gamma=1e-07, score=0.989829, total= 8.8min
[CV] C=10000.0, gamma=1e-06 ..........................................
[CV] ........... C=10000.0, gamma=1e-07, score=0.989293, total= 8.6min
[CV] C=10000.0, gamma=1e-06 ..........................................
[CV] ........... C=10000.0, gamma=1e-07, score=0.987152, total= 8.0min
[CV] C=10000.0, gamma=1e-06 ..........................................
[CV] ........... C=1000.0, gamma=100.0, score=0.476445, total=177.6min
[CV] C=10000.0, gamma=1e-05 ..........................................
[CV] ........... C=10000.0, gamma=1e-06, score=0.987152, total=13.9min
[CV] C=10000.0, gamma=1e-05 ..........................................
[CV] ........... C=10000.0, gamma=1e-06, score=0.989829, total=12.7min
[CV] C=10000.0, gamma=1e-05 ..........................................
[CV] .

[Parallel(n_jobs=15)]: Done 420 tasks      | elapsed: 5156.0min


[CV] .......... C=10000.0, gamma=0.001, score=0.505889, total=184.5min
[CV] C=10000.0, gamma=1.0 ............................................
[CV] .......... C=10000.0, gamma=0.001, score=0.501606, total=178.0min
[CV] C=10000.0, gamma=1.0 ............................................
[CV] .......... C=10000.0, gamma=0.001, score=0.503212, total=204.6min
[CV] C=10000.0, gamma=1.0 ............................................
[CV] .......... C=10000.0, gamma=0.001, score=0.503212, total=204.7min
[CV] C=10000.0, gamma=1.0 ............................................
[CV] .......... C=10000.0, gamma=0.001, score=0.503747, total=220.1min
[CV] C=10000.0, gamma=1.0 ............................................
[CV] ........... C=10000.0, gamma=0.01, score=0.486081, total=203.7min
[CV] C=10000.0, gamma=10.0 ...........................................
[CV] ........... C=10000.0, gamma=0.01, score=0.481799, total=221.4min
[CV] C=10000.0, gamma=10.0 ...........................................
[CV] .

[Parallel(n_jobs=15)]: Done 451 tasks      | elapsed: 5546.4min


[CV] .......... C=10000.0, gamma=100.0, score=0.476445, total=176.3min
[CV] C=100000.0, gamma=1e-07 .........................................
[CV] .......... C=10000.0, gamma=100.0, score=0.475910, total=179.8min
[CV] C=100000.0, gamma=1e-07 .........................................
[CV] .......... C=100000.0, gamma=1e-08, score=0.987687, total= 7.9min
[CV] C=100000.0, gamma=1e-07 .........................................
[CV] .......... C=100000.0, gamma=1e-08, score=0.989293, total= 8.2min
[CV] C=100000.0, gamma=1e-07 .........................................
[CV] .......... C=100000.0, gamma=1e-08, score=0.989293, total= 8.4min
[CV] C=100000.0, gamma=1e-06 .........................................
[CV] .......... C=100000.0, gamma=1e-08, score=0.987152, total= 7.6min
[CV] C=100000.0, gamma=1e-06 .........................................
[CV] .......... C=100000.0, gamma=1e-07, score=0.988758, total= 9.8min
[CV] C=100000.0, gamma=1e-06 .........................................
[CV] .

In [ ]:
print("Best parameters: %s - score: %0.4f" % (top_model.best_params_, top_model.best_score_))

In [ ]:
top_model.cv_results_

In [ ]:
top_model.grid_scores_

In [ ]:
def plot_grid_search(cv_results, grid_param_1, grid_param_2, name_param_1, name_param_2):
    # Get Test Scores Mean and std for each grid search
    scores_mean = cv_results['mean_test_score']
    scores_mean = np.array(scores_mean).reshape(len(grid_param_2),len(grid_param_1))

    scores_sd = cv_results['std_test_score']
    scores_sd = np.array(scores_sd).reshape(len(grid_param_2),len(grid_param_1))

    # Plot Grid search scores
    _, ax = plt.subplots(1,1)

    # Param1 is the X-axis, Param 2 is represented as a different curve (color line)
    for idx, val in enumerate(grid_param_2):
        ax.plot(grid_param_1, scores_mean[idx,:], '-o', label= name_param_2 + ': ' + str(val))

    ax.set_title("Grid Search Scores", fontsize=20, fontweight='bold')
    ax.set_xlabel(name_param_1, fontsize=16)
    ax.set_ylabel('CV Average Score', fontsize=16)
    ax.legend(loc="best", fontsize=15)
    ax.grid('on')
    
    ax.set_xscale('log')
    #ax.set_ylim([0.0,1.0])

In [ ]:
plot_grid_search(top_model.cv_results_,gamma_range,C_range,'Gamma','C')
figure = plt.gcf()
figure.set_size_inches(24, 9)
plt.show()

In [ ]:
skplot.grid_search(top_model.grid_scores_, change='gamma', kind='bar')
figure = plt.gcf()
figure.set_size_inches(24, 9)
plt.show()

In [ ]:
skplot.grid_search(top_model.grid_scores_, change='C', kind='bar')
figure = plt.gcf()
figure.set_size_inches(24, 9)
plt.show()

In [ ]:
cv = StratifiedShuffleSplit(n_splits=5, test_size=0.2, random_state=1)
svm_model = svm.SVC(C=top_model.best_params_['C'],gamma=top_model.best_params_['gamma'])
train_sizes = np.linspace(.1, 1.0, 10)
train_sizes, train_scores, test_scores = learning_curve(svm_model, vgg19features, y, cv=cv, n_jobs=15, train_sizes=train_sizes)

In [ ]:
figure = plt.gcf()
figure.set_size_inches(24, 9)
skplot.learning_curve(train_scores, test_scores, train_sizes)
plt.show()